In [1]:
import mod_sql
import pandas as pd

In [2]:
# mods_sql을 통해서 Database 클래스를 생성
db = mod_sql.Database("localhost", "root", "ubion7", 3306)

In [3]:
sql = """
    select 
    `Country` as '국가', 
    `Units Sold` as '판매개수', 
    `Unit Price` as '가격', 
    `Unit Cost` as '원가', 
    (`Units Sold` * `Unit Price`) as '총수익', 
    (`Units Sold` * `Unit Cost`) as '총비용', 
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from 
    `Sales Records`
"""
db.sql_query(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


In [4]:
df = pd.read_csv('../csv/Sales Records.csv')

In [5]:
df.drop(['Region', 'Item Type', 'Sales Channel', 'Order Priority', 'Order Date', 'Order ID', 'Ship Date'], axis = 1, inplace = True)
#df = df[['Country', 'Units Sold', 'Unit Price', 'Unit Cost']]

In [6]:
df.columns = ['국가', '판매개수', '가격', '원가', '총수익', '총비용', '총이윤']
df

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


In [7]:
df.drop(['총수익', '총비용', '총이윤'], axis = 1, inplace=True)

In [8]:
df['총수익'] = df['판매개수'] * df['가격']
df['총비용'] = df['판매개수'] * df['원가']
df['총이윤'] = (df['판매개수'] * df['가격']) - (df['판매개수'] * df['원가'])
#df['총이윤'] = df['총수익'] - df['총비용']
df

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


### sql에서 반올림

In [9]:
sql = """
    select
    `Total Cost`,
    round(`Total Cost`, 1)
    from
    `Sales Records`
"""

db.sql_query(sql)

,Total Cost,"round(`Total Cost`, 1)"
0,2353920.64,2353920.6
1,34174.25,34174.2
2,592408.95,592409.0
3,3861014.82,3861014.8
4,110978.89,110978.9
...,...,...
9995,273298.63,273298.6
9996,147991.46,147991.5
9997,229507.32,229507.3
9998,3174991.14,3174991.1


## sql 그룹화
1. Country, Item Type을 기준으로 그룹화
2. unit sold의 평균 합계 생성
3. Country를 기준으로 오름차순 정렬

In [10]:
sql = """
    select
    `Country`,
    `Item Type`,
    AVG(`Units Sold`) as AVG,
    SUM(`Units Sold`) as SUM
    from
    `Sales Records`
    group by
    `Country`,
    `Item Type`
    order by
    `Country`
    ASC
"""

db.sql_query(sql)

,Country,Item Type,AVG,SUM
0,Afghanistan,Baby Food,1368.0000,1368
1,Afghanistan,Beverages,4793.3333,14380
2,Afghanistan,Cereal,6101.8889,54917
3,Afghanistan,Clothes,5977.4000,29887
4,Afghanistan,Cosmetics,3044.2500,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.0000,9640
2194,Zimbabwe,Office Supplies,4361.7500,17447
2195,Zimbabwe,Personal Care,9239.0000,18478
2196,Zimbabwe,Snacks,5438.0000,16314


In [11]:
df = pd.read_csv('../csv/Sales Records.csv')

In [12]:
df.groupby(['Country', 'Item Type']).agg(['mean', 'sum'])['Units Sold'].reset_index()

TypeError: Could not convert Middle East and North Africa to numeric

### dept table에서 부서의 이름(dname)이 sales인 소속 사원의 정보 출력
1. dept 테이블에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력

In [13]:
sql = """
    select
    `deptno`
    from
    `dept`
    where
    `dname` = 'sales'
"""

db.sql_query(sql)

,deptno
0,30.0


In [14]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = 30
"""
db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [15]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = (
    select
    `deptno`
    from
    `dept`
    where
    `dname` = 'sales'
    )
"""
db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [16]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = (
        select
        `deptno`
        from
        `dept`
        where
        `dname` = 'research'
    )
"""
db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
2,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
4,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [17]:
sql = """
    select
    `deptno`
    from
    `dept`
    where
    `dname` in ('sales', 'research')
"""

db.sql_query(sql)

,deptno
0,20.0
1,30.0


In [18]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` in (
    select
    `deptno`
    from
    `dept`
    where
    `dname`
    in ('sales', 'research'))
"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
7,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
8,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
9,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [19]:
emp = pd.read_csv('../csv/emp.csv')
dept = pd.read_csv('../csv/dept.csv')

In [20]:
flag = dept.loc[(dept['DNAME'].isin(['RESEARCH', 'SALES'])), 'DEPTNO'].tolist()

In [21]:
emp.loc[emp['DEPTNO'].isin(flag)].reset_index(drop=True)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
7,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
8,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
9,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


1. emp 데이터프레임에서 JOB 컬럼의 데이터를 확인하여 새로운 파생변수 생성
2. job_code SALESMAN이면 1, MANAGER면 2, 나머지는 3으로 하여 새로운 파생변수에 대입

In [22]:
emp['JOB']

0         CLERK
1      SALESMAN
2      SALESMAN
3       MANAGER
4      SALESMAN
5       MANAGER
6       MANAGER
7       ANALYST
8     PRESIDENT
9      SALESMAN
10        CLERK
11        CLERK
12      ANALYST
13        CLERK
Name: JOB, dtype: object

In [23]:
emp['job_code'] = emp['JOB'].apply(lambda x : 1 if (x == 'SALESMAN') else (2 if (x == 'MANAGER') else 3))
emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [24]:
for i in emp.index:
    if emp.loc[i, 'JOB'] == 'SALESMAN':
        emp.loc[i, 'job_code'] = 1
    elif emp.loc[i, 'JOB'] == 'MANAGER':
        emp.loc[i, 'job_code'] = 2
    else:
        emp.loc[i, 'job_code'] = 3
emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [25]:
def change (x):
    if x == 'SALESMAN':
        result = 1
    elif x == 'MANAGER':
        result = 2
    else:
        result = 3
    return result

emp['job_code'] = emp['JOB'].apply(change)
emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1


In [26]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    CASE
        WHEN `JOB` = 'SALESMAN'
        THEN 1
        WHEN `JOB` = 'MANAGER'
        THEN 2
        ELSE 3
    END as 'job_code'
    from
    emp
"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


In [27]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    if(
        `JOB` = 'SALESMAN',
        1,
        if(`JOB` - 'MANAGER',
            2,
            3
        )
    )as `job_code`
    from
    emp 
"""
db.sql_query(sql)

,EMPNO,ENAME,JOB,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,3
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,3
6,7782.0,CLARK,MANAGER,3
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


1. emp 데이터에서 파생변수 H/L 생성
2. SAL이 2000 이상이면 High, 아니면 Low 데이터를 삽입

In [28]:
for i in emp.index:
    if emp.loc[i, 'SAL'] >= 2000 :
        emp.loc[i, 'H/L'] = 'High'
    else:
        emp.loc[i, 'H/L'] = 'Low'

In [29]:
emp['H/L'] = emp['SAL'].apply(lambda x : 'High' if (x >= 2000) else 'Low')

In [30]:
def change (x):
    if x >= 2000:
        result = 'High'
    else:
        result = 'Low'
    return result

emp['H/L'] = emp['SAL'].apply(change)
emp

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1,Low
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1,Low
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2,High
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3,High
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1,Low


In [31]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `SAL`,
    CASE
        WHEN `SAL` >= 2000
        THEN 'High'
        ELSE 'Low'
    END as 'H/L'
    from
    emp
"""

db.sql_query(sql)

,EMPNO,ENAME,SAL,H/L
0,7369.0,SMITH,800.0,Low
1,7499.0,ALLEN,1600.0,Low
2,7521.0,WARD,1250.0,Low
3,7566.0,JONES,2975.0,High
4,7654.0,MARTIN,1250.0,Low
5,7698.0,BLAKE,2850.0,High
6,7782.0,CLARK,2450.0,High
7,7788.0,SCOTT,3000.0,High
8,7839.0,KING,5000.0,High
9,7844.0,TURNER,1500.0,Low


In [32]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `SAL`,
    if(
        `SAL` >= 2000,
        'High',
        'Low'
    )as 'H/L'
    from
    emp 
"""
db.sql_query(sql)

,EMPNO,ENAME,SAL,H/L
0,7369.0,SMITH,800.0,Low
1,7499.0,ALLEN,1600.0,Low
2,7521.0,WARD,1250.0,Low
3,7566.0,JONES,2975.0,High
4,7654.0,MARTIN,1250.0,Low
5,7698.0,BLAKE,2850.0,High
6,7782.0,CLARK,2450.0,High
7,7788.0,SCOTT,3000.0,High
8,7839.0,KING,5000.0,High
9,7844.0,TURNER,1500.0,Low


### join
1. emp 와 dept 테이블 결합

In [33]:
sql = """
    select
    *
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20.0,RESEARCH,DALLAS
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO


In [34]:
sql = """
    select
    emp.EMPNO,
    emp.ENAME,
    emp.DEPTNO,
    dept.DNAME,
    dept.LOC
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

db.sql_query(sql)

,EMPNO,ENAME,DEPTNO,DNAME,LOC
0,7369.0,SMITH,20.0,RESEARCH,DALLAS
1,7499.0,ALLEN,30.0,SALES,CHICAGO
2,7521.0,WARD,30.0,SALES,CHICAGO
3,7566.0,JONES,20.0,RESEARCH,DALLAS
4,7654.0,MARTIN,30.0,SALES,CHICAGO
5,7698.0,BLAKE,30.0,SALES,CHICAGO
6,7782.0,CLARK,10.0,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,20.0,RESEARCH,DALLAS
8,7839.0,KING,10.0,ACCOUNTING,NEW YORK
9,7844.0,TURNER,30.0,SALES,CHICAGO


In [35]:
pd.merge(emp, dept, on='DEPTNO', how='left')


,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L,DNAME,LOC
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low,RESEARCH,DALLAS
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1,Low,SALES,CHICAGO
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1,Low,SALES,CHICAGO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High,RESEARCH,DALLAS
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,1,Low,SALES,CHICAGO
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,2,High,SALES,CHICAGO
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2,High,ACCOUNTING,NEW YORK
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High,RESEARCH,DALLAS
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3,High,ACCOUNTING,NEW YORK
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,1,Low,SALES,CHICAGO


In [36]:
pd.merge(emp, dept, on='DEPTNO', how='right')

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L,DNAME,LOC
0,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,2.0,High,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,3.0,High,ACCOUNTING,NEW YORK
2,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10.0,3.0,Low,ACCOUNTING,NEW YORK
3,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3.0,Low,RESEARCH,DALLAS
4,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2.0,High,RESEARCH,DALLAS
5,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3.0,High,RESEARCH,DALLAS
6,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3.0,Low,RESEARCH,DALLAS
7,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0,3.0,High,RESEARCH,DALLAS
8,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,1.0,Low,SALES,CHICAGO
9,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,1.0,Low,SALES,CHICAGO


### 문제
- 부서의 위치가 'NEW YORK', 'DALLAS'가 아닌 소속 사원의 정보를 출력

In [37]:
sql = """
    select
    *
    from
    dept
    where
    `LOC` not in ('NEW YORK', 'DALLAS')
"""

db.sql_query(sql)

,DEPTNO,DNAME,LOC
0,30.0,SALES,CHICAGO
1,40.0,OPERATIONS,BOSTON


In [38]:
sql = """
    select
    *
    from
    dept
    where
    LOC != 'DALLAS' AND LOC != 'NEW YORK'
"""
db.sql_query(sql)

,DEPTNO,DNAME,LOC
0,30.0,SALES,CHICAGO
1,40.0,OPERATIONS,BOSTON


In [39]:
sql = """
    select
    *
    from
    emp
    where
    DEPTNO = 30
    OR
    DEPTNO = 40
"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [40]:
sql = """
    select
    *
    from
    emp
    where
    DEPTNO in (30, 40)
"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [41]:
sql = """
    select
    *
    from
    emp
    where
    DEPTNO in (
        select
        DEPTNO
        from
        dept
        where
        LOC not in ('NEW YORK', 'DALLAS')
)"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [42]:
sql = """
    select
    *
    from
    emp
    left join
    dept on
    emp.DEPTNO = dept.DEPTNO
    where
    LOC not in('NEW YORK', 'DALLAS'
)"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,30.0,SALES,CHICAGO


### 테이블의 결합
1. 유니언 결합(단순하게 행을 결합)
2. 조인 결합

In [43]:
sql = """
    select
    *
    from
    tran_1
"""
db.sql_query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
4995,T0000005108,210000,2019-06-15 02:42:41,HD315748
4996,T0000005109,150000,2019-06-15 03:36:16,HI215420
4997,T0000005110,50000,2019-06-15 03:44:06,IK880102
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758


In [44]:
sql = """
    select
    *
    from
    tran_2
"""
db.sql_query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000005113,295000,2019-06-15 07:20:27,TS169261
1,T0000005114,50000,2019-06-15 07:35:47,HI599892
2,T0000005115,85000,2019-06-15 07:56:36,HI421757
3,T0000005116,50000,2019-06-15 08:40:55,OA386378
4,T0000005117,120000,2019-06-15 08:44:23,TS506913
...,...,...,...,...
1781,T0000006894,180000,2019-07-31 21:20:44,HI400734
1782,T0000006895,85000,2019-07-31 21:52:48,AS339451
1783,T0000006896,100000,2019-07-31 23:35:25,OA027325
1784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [45]:
sql = """
    select
    *
    from
    tran_1
    union
    select
    *
    from
    tran_2
"""
db.sql_query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
6781,T0000006894,180000,2019-07-31 21:20:44,HI400734
6782,T0000006895,85000,2019-07-31 21:52:48,AS339451
6783,T0000006896,100000,2019-07-31 23:35:25,OA027325
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738


In [46]:
sql = """
    select
    *
    from
    tran_d_1
    union
    select
    *
    from
    tran_d_2
"""
db.sql_query(sql)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [47]:
sql = """
    select
    *
    from
    tran_1
    left join
    tran_d_1
    on tran_1.transaction_id = tran_d_1.transaction_id
"""
db.sql_query(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d_1.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0.0,T0000000113,S005,1.0
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1.0,T0000000114,S001,1.0
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2.0,T0000000115,S003,1.0
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3.0,T0000000116,S005,1.0
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4.0,T0000000117,S002,2.0
...,...,...,...,...,...,...,...,...
5238,T0000005108,210000,2019-06-15 02:42:41,HD315748,NaN,None,None,NaN
5239,T0000005109,150000,2019-06-15 03:36:16,HI215420,NaN,None,None,NaN
5240,T0000005110,50000,2019-06-15 03:44:06,IK880102,NaN,None,None,NaN
5241,T0000005111,210000,2019-06-15 04:14:06,IK074758,NaN,None,None,NaN


In [48]:
sql = """
    select * from
    (select * from tran_1
    union
    select * from tran_2) as tran
    left join
    (select * from tran_d_1
    union
    select * from tran_d_2) as tran_d
    on tran.transaction_id = tran_d.transaction_id
"""

db.sql_query(sql)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006887,210000,2019-07-31 19:43:42,AS961301,7132,T0000006887,S005,1
7140,T0000006888,210000,2019-07-31 20:05:45,GD025393,7133,T0000006888,S005,1
7141,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7142,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1


In [49]:
tran1 = pd.read_csv('../csv/tran_1.csv')
tran2 = pd.read_csv('../csv/tran_2.csv')
tran_d1 = pd.read_csv('../csv/tran_d_1.csv')
tran_d2 = pd.read_csv('../csv/tran_d_2.csv')

In [50]:
df = pd.concat([tran1, tran2], axis=0)

In [51]:
df2 = pd.concat([tran_d1, tran_d2], axis=0)

In [52]:
df3 = pd.merge(df, df2, left_on = 'transaction_id', right_on = 'transaction_id', how = 'left')

In [53]:
df3

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,S002,2
...,...,...,...,...,...,...,...
7139,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,S004,1
7140,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,S002,1
7141,T0000006896,100000,2019-07-31 23:35:25,OA027325,7141,S001,2
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,S002,1


### 문자열이 포함되어 있는 경우 출력

In [54]:
sql = """
    select
    *
    from
    emp
    where
    ENAME like '%%S%%'
"""

db.sql_query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
2,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0
4,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


1. pandas 이용하여 emp.csv에서 ENAME이 S로 시작하는 데이터를 출력
2. s로 끝나는 데이터를 출력
3. s가 포함되어 있는 데이터를 출력

In [55]:
# case1
emp[emp['ENAME'].str.startswith('S')]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High


In [56]:
# case2
data = pd.DataFrame()
for i in emp.index:
    if emp.loc[i, 'ENAME'].startswith('S'):
        data = pd.concat([data, emp.loc[[i]]])

data

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High


In [57]:
#case3
_list = []
for i in emp.index:
    if emp.loc[i, 'ENAME'].startswith('S'):
        _list.append(i)

emp.loc[_list]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High


In [58]:
emp[emp['ENAME'].str.endswith('S')]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low


In [59]:
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') == len(x)-1)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low


In [60]:
emp[emp['ENAME'].str.contains('S')]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low


In [61]:
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') != -1)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low


In [62]:
emp.loc[emp['ENAME'].apply(lambda x: ('S' in x))]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,job_code,H/L
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,3,Low
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,2,High
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,3,High
10,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,3,Low
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,3,Low
